# Python for Finance - part 10

#### 금융분석과 관련된 기본적인 내용을 정리하였습니다. 파이썬을 활용한 금융분석 (이브 힐피시 지음)을 참고하였습니다.

## Chapter 17 파생상품 가치 평가
### 17.1 일반적인 가치 평가 클래스

In [1]:
class valuation_class(object):
    """ 단일 요인 모형의 가치 평가를 위한 베이스 클래스 """
    
    def __init__(self, name, underlying, mar_env, payoff_func=''):
        try:
            self.name = name
            self.pricing_date = mar_envv.pricing_date
            try:
                self.strike = mar_env.get_constant('strike')
            except:
                pass
            self.maturity = mar_env.get_constant('maturity')
            self.currency = mar_env.get_constant('currency')
            self.frequency = underlying.frequency
            self.paths = underlying.paths
            self.discount_curve = underlying.discount_curve
            self.payoff_func = payoff_func
            self.underlying = underlying
            self.underlying.special_dates.extend([self.pricing_date, self.maturity])
            
        except:
            print('Error parsing market environment')
            
    def update(self, initial_value=None, volatility = None, strike=None, maturity=None):
        if initial_value is not None:
            self.underlying.update(initial_value=initial_value)
        if volatility is not None:
            self.underlying.update(volatility=volatility)
        if strike is not None:
            self.strike = strike
        if maturity is not None:
            self.maturity = maturity
            if not maturity in self.underlying.time_grid:
                self.underlying.special_dates.append(maturity)
                self.underlying.instrument_values = None
                
    def delta(self, interval=None, accuracy=4):
        if interval is None:
            interval = self.underlying.initial_value / 50.
            
        value_left = self.present_value(fixed_seed = True)
        initial_del = self.underlying.initial_value + interval
        self.underlying.update(initial_value=initial_del)
        value_right = self.present_value(fixed_seed=True)
        self.underlying.update(initial_value=initial_del-interval)
        delta = (value_right-value_left) / interval
        
        if delta < -1.:
            return -1.
        elif delta > 1.:
            return 1.
        else:
            return round(delta, accuracy)
        
    def vega(self, interval=.01, accuracy=4):
        if interval < self.underlying.volatility / 50.:
            interval = self.underlying.volatility / 50.
        value_left = self.present_value(fixed_seed = True)
        vola_del = self.underlying.volatility + interval
        self.underlying.update(volatility=vola_del)
        value_right = self.present_value(fixed_seed=True)
        self.underlying.update(volatility=vola_del - interval)
        vaga = (value_right - value_left) / interval
        return round(vega, accuracy)

### 17.2 유러피안 행사 방식

In [3]:
class valuation_mcs_european(valuation_class):
    """ 단일 요인 몬테카를로 시뮬레이션을 사용한 임의의 페이오프에 대한 유러피안 옵션 가치 평가 클래스 """
    
    def generate_payoff(self, fixed_seed=False):
        try:
            strike = self.strike
        except:
            pass
        paths = self.underlying.get_instrument_values(fixed_seed=fixed_seed)
        time_grid = self.underlying.time_grid
        
        try:
            time_index = np.where(time_grid == self.maturity)[0]
            time_index = int(time_index)
        except:
            print('Maturity date not in time grid of underlying')
        maturity_value = paths[time_index]
        mean_value = np.mean(paths[:time_index], axis=1)
        max_value = np.amax(paths[:time_index], axis=1)[-1]
        min_value = np.amin(paths[:time_index], axis=1)[-1]
        
        try:
            payoff = eval(self.payoff_func)
            return payoff
        except:
            print("Error evaluating payoff function")
            
    def present_value(self, accuracy=6, fixed_seed=False, full=False):
        cash_flow = self.generate_payoff(fixed_seed=fixed_seed)
        discount_factor = self.discount_curve.get_discount_factors((self.pricing_date, self.maturity))[0,1]
        result = discount_factor * np.sum(cash_flow) / len(cash_flow)
        
        if full:
            return round(result, accuracy), discount_factor*cash_flow
        else:
            return round(result, accuracy)

### 17.3 아메리칸 행사 방식

In [4]:
class valuation_mcs_american(valuation_class):
    """ 단일 요인 몬테카를로 시뮬레이션을 사용한 임의의 페이오프에 대한 아메리칸 옵션 가치 평가 클래스 """
    
    def generate_payoff(self, fixed_seed=False):
        try:
            strike = self.strike
        except:
            pass
        paths = self.underlying.get_instrument_values(fixed_seed=fixed_seed)
        time_grid = self.underlying.time_grid
        
        try:
            time_index_start = int(np.where(time_grid == self.pricing_date)[0])
            time_index_end = int(np.where(time_grid == self.maturity)[0])
        except:
            print('Maturity date not in time grid of underlying')
        
        instrument_values = paths[time_index_start:time_index_end+1]
        
        try:
            payoff = eval(self.payoff_func)
            return instrument_values, payoff, time_index_start, time_index_end
        except:
            print("Error evaluating payoff function")
            
    def present_value(self, accuracy=6, fixed_seed=False, bf=5, full=False):
        
        instrument_values, inner_values, time_index_start, time_index_end = self.generate_payoff(fixed_seed=fixed_seed)
        time_list = self.underlying.time_grid[time_index_start:time_index_end+1]
        discount_factors = self.discount_curve.get_discount_factors(time_list, dtobjects=True)
        V = inner_values[-1]
        
        for t in range(len(time_list) -2, 0, -1):
            df = discount_factors[t, 1] / discount_factors[t+1, t]
            rg = np.polyfit(instrument_values[t], V*df, bf)
            C = np.polyval(rg, instrument_values[t])
            V = np.where(inner_values[t] > C, inner_values[t], V*df)
            
        df = discount_factors[0, 1] / discount_factors[1, 1]
        result = df * np.sum(V) / len(V)
        if full:
            return round(result, accuracy), df * V
        else:
            return round(result, accuracy)